# Lab Exercise: SQL Analysis with Polars

In this lab, you'll practice SQL queries using Polars' built-in SQL functionality. Complete each exercise by writing the appropriate SQL query.

In [6]:
# Setup - Run this cell first
import polars as pl

# Define schema for weather data to ensure 'precip' is read as a float
weather_schema = {
    "origin": pl.Utf8,
    "year": pl.Int64,
    "month": pl.Int64,
    "day": pl.Int64,
    "hour": pl.Int64,
    "temp": pl.Float64,
    "dewp": pl.Float64,
    "humid": pl.Float64,
    "wind_dir": pl.Int64,
    "wind_speed": pl.Float64,
    "wind_gust": pl.Float64,
    "precip": pl.Float64,  # Explicitly set precip to Float64
    "pressure": pl.Float64,
    "visib": pl.Float64,
    "time_hour": pl.Utf8,
}


# Load data
airlines = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_airlines.csv')
airports = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_airports.csv')
flights = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_flights.csv', null_values=['NA'])
planes = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_planes.csv', null_values=['NA'])
weather = pl.read_csv('https://raw.githubusercontent.com/philhetzel/opan5510-class11/refs/heads/main/data/nyc_weather.csv', null_values=['NA'], schema=weather_schema)


flights = flights.with_columns(pl.col("time_hour").str.strptime(pl.Datetime))
weather = weather.with_columns(pl.col("time_hour").str.strptime(pl.Datetime))

# Create SQL context
ctx = pl.SQLContext(
    airlines=airlines,
    airports=airports,
    flights=flights,
    planes=planes,
    weather=weather,
    eager_execution=True
)

print("Setup complete! Tables available:")
print(ctx.execute("SHOW TABLES"))

Setup complete! Tables available:
shape: (5, 1)
┌──────────┐
│ name     │
│ ---      │
│ str      │
╞══════════╡
│ airlines │
│ airports │
│ flights  │
│ planes   │
│ weather  │
└──────────┘


/tmp/ipython-input-1095852092.py:36: DeprecationWarning: The argument `eager_execution` for `SQLContext.__init__` is deprecated. It has been renamed to `eager`.
  ctx = pl.SQLContext(


## Exercise 1: Basic Queries

### 1.1 Find all unique carriers in the airlines table

In [7]:

result1 = ctx.execute("""
    SELECT DISTINCT carrier, name
    FROM airlines
""")

print(result1)

shape: (16, 2)
┌─────────┬────────────────────────┐
│ carrier ┆ name                   │
│ ---     ┆ ---                    │
│ str     ┆ str                    │
╞═════════╪════════════════════════╡
│ 9E      ┆ Endeavor Air Inc.      │
│ AA      ┆ American Airlines Inc. │
│ AS      ┆ Alaska Airlines Inc.   │
│ B6      ┆ JetBlue Airways        │
│ DL      ┆ Delta Air Lines Inc.   │
│ …       ┆ …                      │
│ UA      ┆ United Air Lines Inc.  │
│ US      ┆ US Airways Inc.        │
│ VX      ┆ Virgin America         │
│ WN      ┆ Southwest Airlines Co. │
│ YV      ┆ Mesa Airlines Inc.     │
└─────────┴────────────────────────┘


### 1.2 Find the top 10 destinations by number of flights

In [8]:


result2 = ctx.execute("""
    SELECT dest, COUNT(*) AS num_flights
    FROM flights
    GROUP BY dest
    ORDER BY num_flights DESC
    LIMIT 10
""")

print(result2)

shape: (10, 2)
┌──────┬─────────────┐
│ dest ┆ num_flights │
│ ---  ┆ ---         │
│ str  ┆ u32         │
╞══════╪═════════════╡
│ ORD  ┆ 17283       │
│ ATL  ┆ 17215       │
│ LAX  ┆ 16174       │
│ BOS  ┆ 15508       │
│ MCO  ┆ 14082       │
│ CLT  ┆ 14064       │
│ SFO  ┆ 13331       │
│ FLL  ┆ 12055       │
│ MIA  ┆ 11728       │
│ DCA  ┆ 9705        │
└──────┴─────────────┘


### 1.3 Find all flights that departed more than 2 hours late (120 minutes)

In [9]:
result3 = ctx.execute("""
    SELECT *
    FROM flights
    WHERE dep_delay > 120
""")

print(result3)

shape: (9_723, 19)
┌──────┬───────┬─────┬──────────┬───┬──────────┬──────┬────────┬─────────────────────────┐
│ year ┆ month ┆ day ┆ dep_time ┆ … ┆ distance ┆ hour ┆ minute ┆ time_hour               │
│ ---  ┆ ---   ┆ --- ┆ ---      ┆   ┆ ---      ┆ ---  ┆ ---    ┆ ---                     │
│ i64  ┆ i64   ┆ i64 ┆ i64      ┆   ┆ i64      ┆ i64  ┆ i64    ┆ datetime[μs, UTC]       │
╞══════╪═══════╪═════╪══════════╪═══╪══════════╪══════╪════════╪═════════════════════════╡
│ 2013 ┆ 1     ┆ 1   ┆ 848      ┆ … ┆ 184      ┆ 18   ┆ 35     ┆ 2013-01-01 23:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 957      ┆ … ┆ 200      ┆ 7    ┆ 33     ┆ 2013-01-01 12:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 1114     ┆ … ┆ 1416     ┆ 9    ┆ 0      ┆ 2013-01-01 14:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 1540     ┆ … ┆ 1598     ┆ 13   ┆ 38     ┆ 2013-01-01 18:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 1815     ┆ … ┆ 1134     ┆ 13   ┆ 25     ┆ 2013-01-01 18:00:00 UTC │
│ …    ┆ …     ┆ …   ┆ …        ┆ … ┆ …        ┆ …    ┆ …      ┆ …     

## Exercise 2: Aggregation

### 2.1 Calculate the average departure delay for each origin airport

In [10]:
result4 = ctx.execute("""
    SELECT origin, AVG(dep_delay) AS avg_dep_delay
    FROM flights
    GROUP BY origin
""")

print(result4)

shape: (3, 2)
┌────────┬───────────────┐
│ origin ┆ avg_dep_delay │
│ ---    ┆ ---           │
│ str    ┆ f64           │
╞════════╪═══════════════╡
│ JFK    ┆ 12.112159     │
│ LGA    ┆ 10.346876     │
│ EWR    ┆ 15.107954     │
└────────┴───────────────┘


### 2.2 Find the busiest month of the year

Count the number of flights per month and find which month has the most flights.

In [13]:
result5 = ctx.execute("""
    SELECT *
    FROM flights
    LIMIT 5
""")

print(result5)

result6 = ctx.execute("""
    SELECT month, COUNT(*) AS num_flights
    FROM flights
    GROUP BY month
    ORDER BY num_flights DESC
    LIMIT 1
""")

print(result6)

result7 = ctx.execute("""
    SELECT month, COUNT(*) AS num_flights
    FROM flights
    GROUP BY month
    ORDER BY num_flights DESC
""")

print(result7)

shape: (5, 19)
┌──────┬───────┬─────┬──────────┬───┬──────────┬──────┬────────┬─────────────────────────┐
│ year ┆ month ┆ day ┆ dep_time ┆ … ┆ distance ┆ hour ┆ minute ┆ time_hour               │
│ ---  ┆ ---   ┆ --- ┆ ---      ┆   ┆ ---      ┆ ---  ┆ ---    ┆ ---                     │
│ i64  ┆ i64   ┆ i64 ┆ i64      ┆   ┆ i64      ┆ i64  ┆ i64    ┆ datetime[μs, UTC]       │
╞══════╪═══════╪═════╪══════════╪═══╪══════════╪══════╪════════╪═════════════════════════╡
│ 2013 ┆ 1     ┆ 1   ┆ 517      ┆ … ┆ 1400     ┆ 5    ┆ 15     ┆ 2013-01-01 10:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 533      ┆ … ┆ 1416     ┆ 5    ┆ 29     ┆ 2013-01-01 10:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 542      ┆ … ┆ 1089     ┆ 5    ┆ 40     ┆ 2013-01-01 10:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 544      ┆ … ┆ 1576     ┆ 5    ┆ 45     ┆ 2013-01-01 10:00:00 UTC │
│ 2013 ┆ 1     ┆ 1   ┆ 554      ┆ … ┆ 762      ┆ 6    ┆ 0      ┆ 2013-01-01 11:00:00 UTC │
└──────┴───────┴─────┴──────────┴───┴──────────┴──────┴────────┴───────────

### 2.3 Calculate the on-time performance rate for each carrier

Consider a flight on-time if the departure delay is <= 15 minutes.

In [14]:
result8 = ctx.execute("""
    SELECT carrier,
           AVG(CASE WHEN dep_delay <= 15 THEN 1.0 ELSE 0.0 END) AS on_time_rate
    FROM flights
    GROUP BY carrier
    ORDER BY on_time_rate DESC
""")

print(result8)

shape: (16, 2)
┌─────────┬──────────────┐
│ carrier ┆ on_time_rate │
│ ---     ┆ ---          │
│ str     ┆ f64          │
╞═════════╪══════════════╡
│ HA      ┆ 0.929825     │
│ AS      ┆ 0.865546     │
│ US      ┆ 0.849873     │
│ DL      ┆ 0.830742     │
│ AA      ┆ 0.824376     │
│ …       ┆ …            │
│ FL      ┆ 0.716871     │
│ F9      ┆ 0.715328     │
│ 9E      ┆ 0.701246     │
│ EV      ┆ 0.659221     │
│ YV      ┆ 0.647255     │
└─────────┴──────────────┘


## Exercise 3: Joins

### 3.1 List all flights with their airline names (not just carrier codes)

Show the first 20 flights with carrier code, airline name, flight number, origin, and destination.

In [15]:
result9 = ctx.execute("""
    SELECT f.carrier,
           a.name AS airline_name,
           f.flight,
           f.origin,
           f.dest
    FROM flights f
    JOIN airlines a
      ON f.carrier = a.carrier
    LIMIT 20
""")

print(result9)

shape: (20, 5)
┌─────────┬────────────────────────┬────────┬────────┬──────┐
│ carrier ┆ airline_name           ┆ flight ┆ origin ┆ dest │
│ ---     ┆ ---                    ┆ ---    ┆ ---    ┆ ---  │
│ str     ┆ str                    ┆ i64    ┆ str    ┆ str  │
╞═════════╪════════════════════════╪════════╪════════╪══════╡
│ UA      ┆ United Air Lines Inc.  ┆ 1545   ┆ EWR    ┆ IAH  │
│ UA      ┆ United Air Lines Inc.  ┆ 1714   ┆ LGA    ┆ IAH  │
│ AA      ┆ American Airlines Inc. ┆ 1141   ┆ JFK    ┆ MIA  │
│ B6      ┆ JetBlue Airways        ┆ 725    ┆ JFK    ┆ BQN  │
│ DL      ┆ Delta Air Lines Inc.   ┆ 461    ┆ LGA    ┆ ATL  │
│ …       ┆ …                      ┆ …      ┆ …      ┆ …    │
│ B6      ┆ JetBlue Airways        ┆ 1806   ┆ JFK    ┆ BOS  │
│ UA      ┆ United Air Lines Inc.  ┆ 1187   ┆ EWR    ┆ LAS  │
│ B6      ┆ JetBlue Airways        ┆ 371    ┆ LGA    ┆ FLL  │
│ MQ      ┆ Envoy Air              ┆ 4650   ┆ LGA    ┆ ATL  │
│ B6      ┆ JetBlue Airways        ┆ 343    ┆ EWR    ┆ 

### 3.2 Find the average age of planes for each carrier

Hint: The planes table has a `year` column for manufacture year. Calculate age based on 2013.

In [16]:
result10 = ctx.execute("""
    SELECT f.carrier,
           AVG(2013 - p.year) AS avg_plane_age
    FROM flights f
    JOIN planes p
      ON f.tailnum = p.tailnum
    WHERE p.year IS NOT NULL
    GROUP BY f.carrier
""")

print(result10)

shape: (16, 2)
┌─────────┬───────────────┐
│ carrier ┆ avg_plane_age │
│ ---     ┆ ---           │
│ str     ┆ f64           │
╞═════════╪═══════════════╡
│ VX      ┆ 4.473643      │
│ HA      ┆ 1.548387      │
│ UA      ┆ 13.207691     │
│ WN      ┆ 9.146052      │
│ AS      ┆ 3.33662       │
│ …       ┆ …             │
│ EV      ┆ 11.308998     │
│ F9      ┆ 4.87874       │
│ DL      ┆ 16.372169     │
│ FL      ┆ 11.385829     │
│ YV      ┆ 9.313758      │
└─────────┴───────────────┘


### 3.3 Find flights that experienced both departure delays and bad weather

Join flights with weather data and find flights where departure delay > 30 minutes and either wind_speed > 20 or precip > 0.1

In [17]:
result11 = ctx.execute("""
    SELECT f.year, f.month, f.day, f.hour,
           f.origin, f.dest, f.flight,
           f.dep_delay,
           w.wind_speed, w.precip
    FROM flights f
    JOIN weather w
      ON f.origin = w.origin
     AND f.year = w.year
     AND f.month = w.month
     AND f.day = w.day
     AND f.hour = w.hour
    WHERE f.dep_delay > 30
      AND (w.wind_speed > 20 OR w.precip > 0.1)
""")

print(result11)

shape: (4_938, 10)
┌──────┬───────┬─────┬──────┬───┬────────┬───────────┬────────────┬────────┐
│ year ┆ month ┆ day ┆ hour ┆ … ┆ flight ┆ dep_delay ┆ wind_speed ┆ precip │
│ ---  ┆ ---   ┆ --- ┆ ---  ┆   ┆ ---    ┆ ---       ┆ ---        ┆ ---    │
│ i64  ┆ i64   ┆ i64 ┆ i64  ┆   ┆ i64    ┆ i64       ┆ f64        ┆ f64    │
╞══════╪═══════╪═════╪══════╪═══╪════════╪═══════════╪════════════╪════════╡
│ 2013 ┆ 1     ┆ 1   ┆ 21   ┆ … ┆ 21     ┆ 47        ┆ 21.86482   ┆ 0.0    │
│ 2013 ┆ 1     ┆ 1   ┆ 21   ┆ … ┆ 199    ┆ 116       ┆ 21.86482   ┆ 0.0    │
│ 2013 ┆ 1     ┆ 4   ┆ 11   ┆ … ┆ 4090   ┆ 39        ┆ 20.71404   ┆ 0.0    │
│ 2013 ┆ 1     ┆ 4   ┆ 13   ┆ … ┆ 4231   ┆ 40        ┆ 24.16638   ┆ 0.0    │
│ 2013 ┆ 1     ┆ 4   ┆ 14   ┆ … ┆ 1010   ┆ 33        ┆ 20.71404   ┆ 0.0    │
│ …    ┆ …     ┆ …   ┆ …    ┆ … ┆ …      ┆ …         ┆ …          ┆ …      │
│ 2013 ┆ 9     ┆ 22  ┆ 14   ┆ … ┆ 2347   ┆ 41        ┆ 20.71404   ┆ 0.0    │
│ 2013 ┆ 9     ┆ 22  ┆ 18   ┆ … ┆ 5503   ┆ 97        ┆ 20

## Exercise 4: Advanced Queries

### 4.1 Find the most popular aircraft types (by number of flights)

Join flights with planes to get manufacturer and model information. Show top 10.

In [23]:
result12 = ctx.execute("""
    SELECT
        p.manufacturer || ' ' || p.model AS aircraft_type,
        COUNT(*) AS num_flights
    FROM flights f
    JOIN planes p
      ON f.tailnum = p.tailnum
    GROUP BY p.manufacturer, p.model
    ORDER BY num_flights DESC
    LIMIT 10
""")

print(result12)

shape: (10, 2)
┌─────────────────────────────────┬─────────────┐
│ aircraft_type                   ┆ num_flights │
│ ---                             ┆ ---         │
│ str                             ┆ u32         │
╞═════════════════════════════════╪═════════════╡
│ AIRBUS A320-232                 ┆ 31278       │
│ EMBRAER EMB-145LR               ┆ 28027       │
│ EMBRAER ERJ 190-100 IGW         ┆ 23716       │
│ AIRBUS INDUSTRIE A320-232       ┆ 14553       │
│ EMBRAER EMB-145XR               ┆ 14051       │
│ BOEING 737-824                  ┆ 13809       │
│ BOMBARDIER INC CL-600-2D24      ┆ 11807       │
│ BOEING 737-7H4                  ┆ 10389       │
│ BOEING 757-222                  ┆ 9150        │
│ MCDONNELL DOUGLAS AIRCRAFT CO … ┆ 8932        │
└─────────────────────────────────┴─────────────┘


### 4.2 Analyze route performance

Find the top 10 routes (origin-destination pairs) with:
- Total number of flights
- Average departure delay
- Percentage of flights delayed more than 30 minutes

Include airport names, not just codes.

In [30]:
route_performance = flights.join(
    airports.select(['faa', 'name']),
    left_on='origin',
    right_on='faa',
    how='left'
).rename({'name': 'origin_airport'})

route_performance = route_performance.join(
    airports.select(['faa', 'name']),
    left_on='dest',
    right_on='faa',
    how='left'
).rename({'name': 'dest_airport'})

route_performance = route_performance.group_by(['origin_airport', 'dest_airport']).agg([
    pl.len().alias('total_flights'),
    pl.col('dep_delay').mean().alias('avg_dep_delay'),
    (
        (pl.col('dep_delay') > 30).sum().cast(pl.Float64) / pl.len().cast(pl.Float64) * 100
    ).alias('pct_delayed_over_30')
])

result13_polars = route_performance.sort('total_flights', descending=True).limit(10)

print(result13_polars)

shape: (10, 5)
┌─────────────────────┬──────────────────────┬───────────────┬───────────────┬─────────────────────┐
│ origin_airport      ┆ dest_airport         ┆ total_flights ┆ avg_dep_delay ┆ pct_delayed_over_30 │
│ ---                 ┆ ---                  ┆ ---           ┆ ---           ┆ ---                 │
│ str                 ┆ str                  ┆ u32           ┆ f64           ┆ f64                 │
╞═════════════════════╪══════════════════════╪═══════════════╪═══════════════╪═════════════════════╡
│ John F Kennedy Intl ┆ Los Angeles Intl     ┆ 11262         ┆ 8.522508      ┆ 9.829515            │
│ La Guardia          ┆ Hartsfield Jackson   ┆ 10263         ┆ 11.448621     ┆ 12.247881           │
│                     ┆ Atlanta Int…         ┆               ┆               ┆                     │
│ La Guardia          ┆ Chicago Ohare Intl   ┆ 8857          ┆ 10.740758     ┆ 13.345377           │
│ John F Kennedy Intl ┆ San Francisco Intl   ┆ 8204          ┆ 11.952691    

## Bonus: Compare with Polars

### Choose one of the queries above and implement it using Polars

This will help you understand the relationship between SQL and Polars operations.

In [ ]:
# Example: Let's implement Exercise 2.1 (average delay by origin) in Polars

# SQL version (for reference)
sql_result = ctx.execute("""
    SELECT
        origin,
        AVG(dep_delay) as avg_delay
    FROM flights
    WHERE dep_delay IS NOT NULL
    GROUP BY origin
    ORDER BY avg_delay DESC
""")

# Polars version
polars_result = (
    flights
    .filter(pl.col('dep_delay').is_not_null())
    .group_by('origin')
    .agg(pl.col('dep_delay').mean().alias('avg_delay'))
    .sort('avg_delay', descending=True)
)

print("SQL Result:")
print(sql_result)
print("\nPolars Result:")
print(polars_result)

# Now implement one of your own queries in Polars below:
# Your Polars code here